In [2]:
import pandas as pd
import numpy as np
from sodapy import Socrata
import os
import requests
import matplotlib.pyplot as plt
import pyproj
import datetime
import json
import sys

pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
%matplotlib inline

In [2]:
acs_results_2016 = requests.get('https://api.census.gov/data/2016/acs/acs5?get=NAME,B01001_001E,B01001_011E,B01001_012E,B01001_035E,B01001_036E,B01001_002E,B02001_002E,B02001_003E,B03001_003E,B01002_001E,B25031_001E,B06011_001E,B06009_005E&for=place:*&key=91aa6a44edb890abeea862f2d9befb5d520a5cc4')


In [14]:
# no median gross rent!

acs_results_2009 = requests.get('https://api.census.gov/data/2009/acs5?key=91aa6a44edb890abeea862f2d9befb5d520a5cc4&get=B01001_001E,B01001_011E,B01001_012E,B01001_035E,B01001_036E,B01001_002E,B02001_002E,B02001_003E,B03001_003E,B01002_001E,B06011_001E,B06009_005E,NAME&for=place:*')

In [10]:
acs_results_2009_2 = requests.get('https://api.census.gov/data/2009/acs5?key=YOUR KEY GOES HERE&get=B02001_001E,NAME&for=state:06,36')


In [15]:
acs_results_2009.status_code

200

In [4]:
data = acs_results_2016.text
city_df = pd.read_json(data)
city_df.columns = city_df.iloc[0]
city_df.drop([0], inplace = True) 

In [5]:
city_df.head()

,NAME,B01001_001E,B01001_011E,B01001_012E,B01001_035E,B01001_036E,B01001_002E,B02001_002E,B02001_003E,B03001_003E,B01002_001E,B25031_001E,B06011_001E,B06009_005E,state,place
1,"Abanda CDP, Alabama",151,0,24,18,0,71,138,13,0,32.9,-666666666,12908,0,01,00100
2,"Abbeville city, Alabama",2627,81,72,52,41,1359,1386,1211,39,50.4,514,17653,116,01,00124
3,"Adamsville city, Alabama",4422,168,95,63,91,2164,2147,2243,28,42.0,844,26402,301,01,00460
4,"Addison town, Alabama",757,18,41,26,11,383,722,0,0,46.2,471,20765,64,01,00484
5,"Akron town, Alabama",252,0,6,8,0,121,36,216,0,43.9,533,11875,5,01,00676


In [6]:
el_dorado = requests.get('https://api.census.gov/data/2016/acs/acs5?get=NAME,B01001_001E,B01001_011E,B01001_012E,B01001_035E,B01001_036E,B01001_002E,B02001_002E,B02001_003E,B03001_003E,B01002_001E,B25031_001E,B06011_001E,B06009_005E&for=county:017&in=state:06&key=91aa6a44edb890abeea862f2d9befb5d520a5cc4')


In [7]:
ed_data = el_dorado.text
ed_df = pd.read_json(ed_data)
ed_df.columns = ed_df.iloc[0]
ed_df.drop([0], inplace = True) 
ed_df.rename(columns={'county':'place'}, inplace=True)


In [10]:
mb = requests.get('https://api.census.gov/data/2016/acs/acs5?get=NAME,B01001_001E,B01001_011E,B01001_012E,B01001_035E,B01001_036E,B01001_002E,B02001_002E,B02001_003E,B03001_003E,B01002_001E,B25031_001E,B06011_001E,B06009_005E&for=county%20subdivision:40850&in=state:25%20county:023&key=91aa6a44edb890abeea862f2d9befb5d520a5cc4')

mb.status_code
mb_data = mb.text
print(mb_data)

[["NAME","B01001_001E","B01001_011E","B01001_012E","B01001_035E","B01001_036E","B01001_002E","B02001_002E","B02001_003E","B03001_003E","B01002_001E","B25031_001E","B06011_001E","B06009_005E","state","county","county subdivision"],
["Middleborough town, Plymouth County, Massachusetts","24042","515","828","918","892","12088","22763","422","296","43.0","1111","36214","3413","25","023","40850"]]


In [18]:
mb = requests.get('https://api.census.gov/data/2016/acs/acs5?get=NAME,B01001_001E,B01001_011E,B01001_012E,B01001_035E,B01001_036E,B01001_002E,B02001_002E,B02001_003E,B03001_003E,B01002_001E,B25031_001E,B06011_001E,B06009_005E&for=county%20subdivision:40850&in=state:25%20county:023&key=91aa6a44edb890abeea862f2d9befb5d520a5cc4')
mb_data = mb.text
mb_df = pd.read_json(mb_data)
mb_df.columns = mb_df.iloc[0]
mb_df.drop([0], inplace = True)
mb_df['place'] = mb_df['county'] + mb_df['county subdivision']
mb_df.drop(columns = ['county', 'county subdivision'], inplace = True)

In [19]:
boulder = requests.get('https://api.census.gov/data/2016/acs/acs5?get=NAME,B01001_001E,B01001_011E,B01001_012E,B01001_035E,B01001_036E,B01001_002E,B02001_002E,B02001_003E,B03001_003E,B01002_001E,B25031_001E,B06011_001E,B06009_005E&for=county:013&in=state:08&key=91aa6a44edb890abeea862f2d9befb5d520a5cc4')
b_data = boulder.text
b_df = pd.read_json(b_data)
b_df.columns = b_df.iloc[0]
b_df.drop([0], inplace = True) 
b_df.rename(columns={'county':'place'}, inplace=True)
total_df = pd.concat([city_df, ed_df, b_df, mb_df])

In [20]:
total_df.tail()

,NAME,B01001_001E,B01001_011E,B01001_012E,B01001_035E,B01001_036E,B01001_002E,B02001_002E,B02001_003E,B03001_003E,B01002_001E,B25031_001E,B06011_001E,B06009_005E,state,place
29573,"Yauco zona urbana, Puerto Rico",17627,511,605,504,650,8080,14877,458,17562,37.0,408,None,None,72,88035
29574,"Yaurel comunidad, Puerto Rico",1255,0,33,27,80,580,635,395,1255,36.0,-666666666,None,None,72,88121
1,"El Dorado County, California",183000,4543,4617,4118,4554,91349,159165,1784,22868,45.2,1105,31086,27316,06,017
1,"Boulder County, Colorado",313961,11803,10105,9852,9696,157617,278048,2914,42914,36.0,1236,31927,64270,08,013
1,"Middleborough town, Plymouth County, Massachus...",24042,515,828,918,892,12088,22763,422,296,43.0,1111,36214,3413,25,02340850


In [21]:
variables = {'B01001_001E':"population",'B01001_011E':"m_25_29", 'B01001_012E':'m_30_34', 
             'B01001_035E': "f_25_29", 'B01001_036E':"f_30_34",'B01001_002E':"total_male",'DP05_0066PE': "pct_hispanic",
             'B02001_002E': "total_white",'B02001_003E':"total_black",'B03001_003E': "total_hispanic", 'B01002_001E':"median_age",'B25031_001E':"median_gross_rent",
             'B06011_001E': "median_income",'B06009_005E':"total_bachelor"}
total_df.rename(columns = variables, inplace=True)
total_df['year'] = 2016


In [22]:
cols = ['population', 'f_25_29', 'm_25_29', 'f_30_34', 'm_30_34', 'total_male', 'total_black', 'total_white', 
       'total_hispanic', 'total_bachelor']
total_df[cols] = total_df[cols].apply(pd.to_numeric, errors='coerce')

total_df['pct_25_34'] = (total_df['f_25_29'] + total_df['m_25_29'] + total_df['f_30_34']+ total_df['f_30_34'])/total_df['population']
total_df['pct_male'] = total_df['total_male']/total_df['population']
total_df['pct_black'] = total_df['total_black']/total_df['population']
total_df['pct_white'] = total_df['total_white']/total_df['population']
total_df['pct_hispanic'] = total_df['total_hispanic']/total_df['population']
total_df['pct_bachelor'] = total_df['total_bachelor']/total_df['population']

In [23]:
cs = total_df['NAME'].str.split(",", expand = True)
total_df['city'] = cs[0]
total_df['state_name'] = cs[1]

In [24]:
st = pd.read_table('state_code.txt', sep = '|')

In [25]:
st.head()

,STATE,STUSAB,STATE_NAME,STATENS
0,1,AL,Alabama,1779775
1,2,AK,Alaska,1785533
2,4,AZ,Arizona,1779777
3,5,AR,Arkansas,68085
4,6,CA,California,1779778


In [26]:
st.rename(index = str, columns = {'STATE': 'state','STUSAB':'state_code', 'STATE_NAME': 'state_name'}, inplace = True)
st['state'] = st['state'].apply(lambda x: '{0:0>2}'.format(x))
st = st[['state', 'state_code']]

In [27]:
total_df = total_df.merge(st, how = 'left', on = 'state')

In [53]:
total_df[total_df['city'] == 'Riverside city']

,NAME,population,m_25_29,m_30_34,f_25_29,f_30_34,total_male,total_white,total_black,total_hispanic,median_age,median_gross_rent,median_income,total_bachelor,state,place,year,pct_25_34,pct_male,pct_black,pct_white,pct_hispanic,pct_bachelor,city,state_name,state_code
3074,"Riverside city, California",318678,12870,12352,12534,11066,158097,206638,19164,165670,31.4,1194,24420,25444.0,06,62000,2016,0.149166,0.496103,0.060136,0.648423,0.519866,0.079842,Riverside city,California,CA
8877,"Riverside city, Iowa",1054,61,14,64,13,474,1036,3,4,42.2,701,26960,77.0,19,67440,2016,0.143264,0.449715,0.002846,0.982922,0.003795,0.073055,Riverside city,Iowa,IA
14456,"Riverside city, Missouri",3103,87,210,85,183,1625,2360,284,166,36.7,642,19115,321.0,29,62156,2016,0.173381,0.523687,0.091524,0.760554,0.053497,0.103448,Riverside city,Missouri,MO
20099,"Riverside city, Ohio",25036,1237,776,975,749,12681,21466,1947,990,35.7,798,25958,1875.0,39,67468,2016,0.148187,0.506511,0.077768,0.857405,0.039543,0.074892,Riverside city,Ohio,OH
25857,"Riverside city, Texas",495,0,11,9,6,219,363,121,10,53.1,834,27857,33.0,48,62408,2016,0.042424,0.442424,0.244444,0.733333,0.020202,0.066667,Riverside city,Texas,TX


In [29]:
total_df.to_csv('city_data.csv', index= False)

In [28]:
total_df.tail()

,NAME,population,m_25_29,m_30_34,f_25_29,f_30_34,total_male,total_white,total_black,total_hispanic,median_age,median_gross_rent,median_income,total_bachelor,state,place,year,pct_25_34,pct_male,pct_black,pct_white,pct_hispanic,pct_bachelor,city,state_name,state_code
29572,"Yauco zona urbana, Puerto Rico",17627,511,605,504,650,8080,14877,458,17562,37.0,408,None,NaN,72,88035,2016,0.131333,0.458388,0.025983,0.843989,0.996312,NaN,Yauco zona urbana,Puerto Rico,PR
29573,"Yaurel comunidad, Puerto Rico",1255,0,33,27,80,580,635,395,1255,36.0,-666666666,None,NaN,72,88121,2016,0.149004,0.462151,0.314741,0.505976,1.000000,NaN,Yaurel comunidad,Puerto Rico,PR
29574,"El Dorado County, California",183000,4543,4617,4118,4554,91349,159165,1784,22868,45.2,1105,31086,27316.0,06,017,2016,0.097098,0.499175,0.009749,0.869754,0.124962,0.149268,El Dorado County,California,CA
29575,"Boulder County, Colorado",313961,11803,10105,9852,9696,157617,278048,2914,42914,36.0,1236,31927,64270.0,08,013,2016,0.130739,0.502027,0.009281,0.885613,0.136686,0.204707,Boulder County,Colorado,CO
29576,"Middleborough town, Plymouth County, Massachus...",24042,515,828,918,892,12088,22763,422,296,43.0,1111,36214,3413.0,25,02340850,2016,0.133808,0.502787,0.017553,0.946801,0.012312,0.141960,Middleborough town,Plymouth County,MA


In [21]:
# 2009 Census Data
acs_results_2009 = requests.get('https://api.census.gov/data/2009/acs5?key=91aa6a44edb890abeea862f2d9befb5d520a5cc4&get=B01001_001E,B01001_011E,B01001_012E,B01001_035E,B01001_036E,B01001_002E,B02001_002E,B02001_003E,B03001_003E,B01002_001E,B06011_001E,B06009_005E,NAME&for=place:*')
data = acs_results_2009.text
city_df = pd.read_json(data)
city_df.columns = city_df.iloc[0]
city_df.drop([0], inplace = True) 

el_dorado_09 = requests.get('https://api.census.gov/data/2009/acs5?key=91aa6a44edb890abeea862f2d9befb5d520a5cc4&get=NAME,B01001_001E,B01001_011E,B01001_012E,B01001_035E,B01001_036E,B01001_002E,B02001_002E,B02001_003E,B03001_003E,B01002_001E,B06011_001E,B06009_005E&for=county:017&in=state:06')
ed_data = el_dorado_09.text
ed_df = pd.read_json(ed_data)
ed_df.columns = ed_df.iloc[0]
ed_df.drop([0], inplace = True) 
ed_df.rename(columns={'county':'place'}, inplace=True)

mb_09 = requests.get('https://api.census.gov/data/2009/acs5?key=91aa6a44edb890abeea862f2d9befb5d520a5cc4&get=NAME,B01001_001E,B01001_011E,B01001_012E,B01001_035E,B01001_036E,B01001_002E,B02001_002E,B02001_003E,B03001_003E,B01002_001E,B06011_001E,B06009_005E&for=county%20subdivision:40850&in=state:25%20county:023')
mb_data = mb_09.text
mb_df = pd.read_json(mb_data)
mb_df.columns = mb_df.iloc[0]
mb_df.drop([0], inplace = True)
mb_df['place'] = mb_df['county'] + mb_df['county subdivision']
mb_df.drop(columns = ['county', 'county subdivision'], inplace = True)

boulder_09 = requests.get('https://api.census.gov/data/2009/acs5?key=91aa6a44edb890abeea862f2d9befb5d520a5cc4&get=NAME,B01001_001E,B01001_011E,B01001_012E,B01001_035E,B01001_036E,B01001_002E,B02001_002E,B02001_003E,B03001_003E,B01002_001E,B06011_001E,B06009_005E&for=county:013&in=state:08')
b_data = boulder_09.text
b_df = pd.read_json(b_data)
b_df.columns = b_df.iloc[0]
b_df.drop([0], inplace = True) 
b_df.rename(columns={'county':'place'}, inplace=True)
total_df = pd.concat([city_df, ed_df, b_df, mb_df])


variables = {'B01001_001E':"population_09",'B01001_011E':"m_25_29", 'B01001_012E':'m_30_34', 
             'B01001_035E': "f_25_29", 'B01001_036E':"f_30_34",'B01001_002E':"total_male",
             'B02001_002E': "total_white",'B02001_003E':"total_black",'B03001_003E': "total_hispanic", 'B01002_001E':"median_age_09",
             'B06011_001E': "median_income_09",'B06009_005E':"total_bachelor"}
total_df.rename(columns = variables, inplace=True)
total_df['year'] = 2009

In [18]:
total_df.head()

,population,total_male,m_25_29,m_30_34,f_25_29,f_30_34,median_age,total_white,total_black,total_hispanic,total_bachelor,median_income,NAME,place,state,year
1,64,36,0,4,1,6,39.5,40,0,10,7,35833,"Adak city, Alaska",00065,02,2009
2,101,69,11,0,2,6,29.6,10,0,22,7,8688,"Akhiok city, Alaska",00650,02,2009
3,579,309,3,27,19,11,26.3,22,0,0,9,11667,"Akiachak CDP, Alaska",00760,02,2009
4,289,137,22,8,15,13,26.9,16,0,0,12,11058,"Akiak city, Alaska",00870,02,2009
5,1210,813,139,139,49,8,37.6,77,275,144,116,20996,"Akutan city, Alaska",01090,02,2009


In [26]:
cols = ['population_09', 'f_25_29', 'm_25_29', 'f_30_34', 'm_30_34', 'total_male', 'total_black', 'total_white', 
       'total_hispanic', 'total_bachelor']
total_df[cols] = total_df[cols].apply(pd.to_numeric, errors='coerce')

total_df['pct_25_34_09'] = (total_df['f_25_29'] + total_df['m_25_29'] + total_df['f_30_34']+ total_df['f_30_34'])/total_df['population_09']
total_df['pct_male_09'] = total_df['total_male']/total_df['population_09']
total_df['pct_black_09'] = total_df['total_black']/total_df['population_09']
total_df['pct_white_09'] = total_df['total_white']/total_df['population_09']
total_df['pct_hispanic_09'] = total_df['total_hispanic']/total_df['population_09']
total_df['pct_bachelor_09'] = total_df['total_bachelor']/total_df['population_09']

cols_to_keep = ['population_09', 'median_age_09', 'median_income_09', 'NAME', 'place', 'state', 'year', 'pct_25_34_09', 
                'pct_male_09', 'pct_black_09', 'pct_white_09', 'pct_hispanic_09', 'pct_bachelor_09']

total_df = total_df[cols_to_keep]

In [27]:
cs = total_df['NAME'].str.split(",", expand = True)
total_df['city'] = cs[0]
total_df['state_name'] = cs[1]
st = pd.read_table('state_code.txt', sep = '|')
st.rename(index = str, columns = {'STATE': 'state','STUSAB':'state_code', 'STATE_NAME': 'state_name'}, inplace = True)
st['state'] = st['state'].apply(lambda x: '{0:0>2}'.format(x))
st = st[['state', 'state_code']]
total_df = total_df.merge(st, how = 'left', on = 'state')
total_df.to_csv('city_data_2009.csv', index= False)

/Users/alenastern/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/alenastern/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [28]:
total_df.head()

,population_09,median_age_09,median_income_09,NAME,place,state,year,pct_25_34_09,pct_male_09,pct_black_09,pct_white_09,pct_hispanic_09,pct_bachelor_09,city,state_name,state_code
0,64,39.5,35833,"Adak city, Alaska",00065,02,2009,0.203125,0.562500,0.000000,0.625000,0.156250,0.109375,Adak city,Alaska,AK
1,101,29.6,8688,"Akhiok city, Alaska",00650,02,2009,0.247525,0.683168,0.000000,0.099010,0.217822,0.069307,Akhiok city,Alaska,AK
2,579,26.3,11667,"Akiachak CDP, Alaska",00760,02,2009,0.075993,0.533679,0.000000,0.037997,0.000000,0.015544,Akiachak CDP,Alaska,AK
3,289,26.9,11058,"Akiak city, Alaska",00870,02,2009,0.217993,0.474048,0.000000,0.055363,0.000000,0.041522,Akiak city,Alaska,AK
4,1210,37.6,20996,"Akutan city, Alaska",01090,02,2009,0.168595,0.671901,0.227273,0.063636,0.119008,0.095868,Akutan city,Alaska,AK
